<a href="https://colab.research.google.com/github/Arjunkalliyadath/Github-Project/blob/main/Data_Acquisition_Case_Study_08_12_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
import pandas as pd
import sqlite3
import numpy as np

# Step 1: Load SpaceX Launch Data from API

Use https://api.spacexdata.com/v4/launches

In [201]:
launch_Data = "https://api.spacexdata.com/v4/launches"
launches_df = pd.read_json(launch_Data)

In [202]:
launches_df.head()

,fairings,links,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,...,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,...,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,...,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,0.0,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,...,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c
3,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],Ratsat was carried to orbit on the first succe...,...,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d
4,"{'reused': False, 'recovery_attempt': False, '...",{'patch': {'small': 'https://images2.imgbox.co...,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,1.0,[],None,...,2009-07-13T03:35:00.000Z,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e


In [203]:
launches_df.shape

(205, 27)

In [204]:
launches_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fairings               169 non-null    object 
 1   links                  205 non-null    object 
 2   static_fire_date_utc   121 non-null    object 
 3   static_fire_date_unix  121 non-null    float64
 4   net                    205 non-null    bool   
 5   window                 117 non-null    float64
 6   rocket                 205 non-null    object 
 7   success                186 non-null    float64
 8   failures               205 non-null    object 
 9   details                134 non-null    object 
 10  crew                   205 non-null    object 
 11  ships                  205 non-null    object 
 12  capsules               205 non-null    object 
 13  payloads               205 non-null    object 
 14  launchpad              205 non-null    object 
 15  flight

Extract relevant columns: name, date_utc, success, details, rocket

In [205]:
launches_clean = launches_df[["name","date_utc","success","details","rocket"]].copy()
launches_clean.head()

,name,date_utc,success,details,rocket
0,FalconSat,2006-03-24T22:30:00.000Z,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb
1,DemoSat,2007-03-21T01:10:00.000Z,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb
2,Trailblazer,2008-08-03T03:34:00.000Z,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb
3,RatSat,2008-09-28T23:15:00.000Z,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb
4,RazakSat,2009-07-13T03:35:00.000Z,1.0,None,5e9d0d95eda69955f709d1eb


Convert date_utc to datetime and extract the year

In [206]:
launches_clean["date_utc"] = pd.to_datetime(launches_clean["date_utc"])
launches_clean["year"] = launches_clean["date_utc"].dt.year

# Step 2: Load Rocket Metadata

Use https://api.spacexdata.com/v4/rockets


In [207]:
rockets_df = pd.read_json("https://api.spacexdata.com/v4/rockets")
rockets_df.head()

,height,diameter,mass,first_stage,second_stage,engines,landing_legs,payload_weights,flickr_images,name,...,stages,boosters,cost_per_launch,success_rate_pct,first_flight,country,company,wikipedia,description,id
0,"{'meters': 22.25, 'feet': 73}","{'meters': 1.6800000000000002, 'feet': 5.5}","{'kg': 30146, 'lb': 66460}","{'thrust_sea_level': {'kN': 420, 'lbf': 94000}...","{'thrust': {'kN': 31, 'lbf': 7000}, 'payloads'...","{'isp': {'sea_level': 267, 'vacuum': 304}, 'th...","{'number': 0, 'material': None}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...","[https://imgur.com/DaCfMsj.jpg, https://imgur....",Falcon 1,...,2,0,6700000,40,2006-03-24,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb
1,"{'meters': 70, 'feet': 229.6}","{'meters': 3.7, 'feet': 12}","{'kg': 549054, 'lb': 1207920}","{'thrust_sea_level': {'kN': 7607, 'lbf': 17100...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 4, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm1.staticflickr.com/929/2878733830...,Falcon 9,...,2,0,50000000,98,2010-06-04,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec
2,"{'meters': 70, 'feet': 229.6}","{'meters': 12.2, 'feet': 39.9}","{'kg': 1420788, 'lb': 3125735}","{'thrust_sea_level': {'kN': 22819, 'lbf': 5130...","{'thrust': {'kN': 934, 'lbf': 210000}, 'payloa...","{'isp': {'sea_level': 288, 'vacuum': 312}, 'th...","{'number': 12, 'material': 'carbon fiber'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://farm5.staticflickr.com/4599/385838292...,Falcon Heavy,...,2,2,90000000,100,2018-02-06,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed
3,"{'meters': 118, 'feet': 387}","{'meters': 9, 'feet': 30}","{'kg': 1335000, 'lb': 2943000}","{'thrust_sea_level': {'kN': 128000, 'lbf': 287...","{'thrust': {'kN': 1957, 'lbf': 440000}, 'paylo...","{'isp': {'sea_level': 330, 'vacuum': 380}, 'th...","{'number': 6, 'material': 'stainless steel'}","[{'id': 'leo', 'name': 'Low Earth Orbit', 'kg'...",[https://live.staticflickr.com/65535/489541389...,Starship,...,2,0,7000000,0,2021-12-01,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee


Extract id, name, type, active, and stages


In [208]:
rockets_clean = rockets_df[["id", "name", "type", "active", "stages"]]
rockets_clean.head()

,id,name,type,active,stages
0,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
1,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2
3,5e9d0d96eda699382d09d1ee,Starship,rocket,False,2


# Step 3: Merge Launch and Rocket Data

Join the two DataFrames on rocket ID using

In [209]:
merged_df = pd.DataFrame(launches_clean.merge(rockets_clean, left_on="rocket", right_on="id"))
merged_df

,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
...,...,...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2


# Step 4: Add Simulated Country Information

Add a new column country and randomly assign one of these values:
['USA', 'Russia', 'India', 'China', 'France']

In [210]:
countries = ['USA', 'Russia', 'India', 'China', 'France']
merged_df["country"] = np.random.choice(countries, size=len(merged_df))

In [211]:
merged_df

,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages,country
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,China
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,USA
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,India
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,India
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,China
...,...,...,...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,USA
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,China
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,France
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2,Russia


# Step 5: Store Merged Data in SQLite3

Use sqlite3 to create a connection and save the merged DataFrame as a table named
launches

In [212]:
conn = sqlite3.connect("spacex.db")

Table should contain all merged columns including country

In [213]:
merged_df.to_sql("launches", conn, if_exists="replace", index=False)
merged_df

,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages,country
0,FalconSat,2006-03-24 22:30:00+00:00,0.0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,China
1,DemoSat,2007-03-21 01:10:00+00:00,0.0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,USA
2,Trailblazer,2008-08-03 03:34:00+00:00,0.0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,India
3,RatSat,2008-09-28 23:15:00+00:00,1.0,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,India
4,RazakSat,2009-07-13 03:35:00+00:00,1.0,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,China
...,...,...,...,...,...,...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,USA
201,TTL-1,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,China
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69973a809d1ec,2022,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2,France
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,NaN,None,5e9d0d95eda69974db09d1ed,2022,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2,Russia


# Step 6: Run SQL Queries on the Data to analyze


Launches by Country

In [214]:
Lauches_by_country = """
SELECT country, COUNT(*) AS total_launches
FROM launches
GROUP BY country;
"""

pd.read_sql_query(Lauches_by_country, conn)

,country,total_launches
0,China,51
1,France,36
2,India,46
3,Russia,42
4,USA,30


Which year had the highest number of launches?

In [218]:
Highest_no_of_launches = """
SELECT year, COUNT(*) AS total_launches
FROM launches
GROUP BY year
ORDER BY total_launches DESC
LIMIT 1;
"""

pd.read_sql_query(Highest_no_of_launches, conn)

,year,total_launches
0,2022,62


Top 5 Missions by Launch Count

In [217]:
Top5Missions = """
SELECT name_x AS mission_name,
       COUNT(*) AS launch_count
FROM launches
GROUP BY name_x
ORDER BY launch_count DESC;
"""

pd.read_sql_query(Top5Missions, conn)

,mission_name,launch_count
0,ispace Mission 1 & Rashid,1
1,ZUMA,1
2,WorldView Legion 1 & 2,1
3,Viasat-3 & Arcturus,1
4,USSF-44,1
...,...,...
200,Amos-6,1
201,Amos-17,1
202,ANASIS-II,1
203,ABS-3A / Eutelsat 115W B,1


Top Rocket by Launch Count

In [228]:
Top1Rocket = """
SELECT name_y AS Rocket_name,
       COUNT(*) AS launch_count
FROM launches
GROUP BY name_y
ORDER BY launch_count DESC LIMIT 1
"""

pd.read_sql_query(Top1Rocket, conn)

,Rocket_name,launch_count
0,Falcon 9,195
